## Multinomial Event Model

In [1]:
x = ["This was an awesome movie!",
    "Great Movie. I liked it a lot.",
    "Happy Ending. Awesome acting by the hero.",
    "loved it. truly great.",
    "bad, not upto the mark.",
    "could have been better."
    "surely a disappointing movie"]

In [2]:
y = [1,1,1,1,0,0,0] # 1 stands for a positive review, 0 stands for negative review.

In [3]:
## To do: Given these reviews and their class i.e. positive or negative, predict class for a new review.

In [4]:
# testing data
x_test = ["I was happy happy and I loved the acting in the movie",
         "The movie I saw was bad"]

## 1. Cleaning:

In [7]:
from nltk.tokenize import RegexpTokenizer #import regular expression tokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [8]:
# Init objects
tokenizer = RegexpTokenizer(r'\w+') # r stands for regular expression and '\w+' stands for all the words.
en_stopwords = set(stopwords.words('english'))
ps = PorterStemmer()

In [9]:
def getCleanReview(review):
    review = review.lower()
    review = review.replace("<br /><br />"," ") # there are many break tags as the training data is extracted from some html page
    # replaced all break tags with space.
    #Tokenize
    tokens = tokenizer.tokenize(review)
    new_tokens = [token for token in tokens if token not in en_stopwords]
    stemmed_tokens = [ps.stem(token) for token in new_tokens]
    
    cleaned_review = ' '.join(stemmed_tokens)
    
    return cleaned_review